In [1]:
# Import libraries for simulation
import tensorflow as tf
import numpy as np
import random as r
import datetime as dt

/home/ruben/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
dimensions = (8,8)
mineProbability = 0.2      # Probability that a square contain a mine
missingProbability = 0.1   # Probability that a square is missing adjacency info

In [3]:
# count the number of mines in the proximity of given square, including square itself
def countMines(board,r,c):
    count = 0
    rows, cols = board.shape
    for i in [r-1,r,r+1]:
        if i >= 0 and i < rows:
            for j in [c-1,c,c+1]:
                if j >= 0 and j < cols:
                    count += int(board[i,j])
    return count

In [4]:
def minesweepMatrix(dimensions):
    rows,cols = dimensions
    size = rows * cols
    A = np.zeros([size,size],dtype=int)
    for rA in range(size):
        for cA in range(size):
            inRow, inCol = divmod(rA,cols)
            outRow, outCol = divmod(cA,cols)
            A[rA,cA] = abs(inRow-outRow) <= 1 and abs(inCol-outCol) <= 1
    return(A)

In [5]:
# Converts a board of mines into a board of mine counts
'''
def boardMineCounts_(board):
    mineInfo = np.zeros(board.shape, dtype = int)
    rows, cols = board.shape
    for i in range(rows):
        for j in range(cols):
            mineInfo[i,j] = countMines(board,i,j)
    return mineInfo
'''
def boardMineCounts(board):
    return(minesweepMatrix(board.shape).dot(board.flatten()).reshape(board.shape))

In [6]:
def boardPartialMineCounts(board):
    result = boardMineCounts(board)
    for index, x in np.ndenumerate(board):
        if x: result[index] = -1
        elif r.uniform(0, 1) < missingProbability: result[index] = -1
    return result

In [7]:
# Generates a random training batch of size at most n
def next_training_batch(n):
    batch_xs = []
    batch_ys = []
    for _ in range(n):
        board = np.random.random(dimensions) < mineProbability
        counts = boardPartialMineCounts(board)
        frees = (1 - board).flatten().astype(float)
        freesSum = sum(frees)
        if freesSum > 0:
            batch_xs.append(counts.flatten())
            batch_ys.append(frees / freesSum)
    return (np.asarray(batch_xs), np.asarray(batch_ys))

In [8]:
# Create the model
rows, cols = dimensions
size = rows*cols
mineCounts = tf.placeholder(tf.int32, [None, size], name="mineCounts")
mineCountsOneHot = tf.reshape(tf.one_hot(mineCounts+1,10), [-1, size*10])
W = tf.Variable(tf.random_normal([size*10, size], stddev=0.01), name="W")
b = tf.Variable(tf.random_normal([size], stddev=0.01), name="b")
y = tf.matmul(mineCountsOneHot, W) + b

In [9]:
mineFreeAverages = tf.placeholder(tf.float32, [None, size], name="mineFreeAverages")

In [10]:
# Loss function
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=mineFreeAverages, logits=y))

In [11]:
# Summaries for tensorboard
with tf.name_scope('W_reshape'):
    image_shaped_W = tf.reshape(W, [-1, size*10, size, 1])
    tf.summary.image('W', image_shaped_W, 1000)

with tf.name_scope('b_reshape'):
    image_shaped_b = tf.reshape(b, [-1, rows, cols, 1])
    tf.summary.image('b', image_shaped_b, 1000)

_ = tf.summary.scalar('accuracy', cross_entropy)

In [12]:
# Optimiser
train_step = tf.train.AdamOptimizer().minimize(cross_entropy)

In [13]:
# Create session and initialise or restore stuff
savePath = './saves.tf.Mines3/' + str(dimensions) + '/'
saver = tf.train.Saver()

sess = tf.InteractiveSession()

merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('.', sess.graph)

In [14]:
tf.global_variables_initializer().run()

In [15]:
# Restore model?
#saver.restore(sess, savePath + "model-10000")

In [16]:
# Train
for iteration in range(4001):
    batch_xs, batch_ys = next_training_batch(100)
    if iteration % 10 == 0:
        summary, acc, _ = sess.run([merged, cross_entropy, train_step],
                                   feed_dict={mineCounts: batch_xs, mineFreeAverages: batch_ys})
        writer.add_summary(summary, iteration)
        print('%s: Accuracy at step %s: %s' % (dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), iteration, acc))
    else:
        _ = sess.run(train_step, feed_dict={mineCounts: batch_xs, mineFreeAverages: batch_ys})
    if iteration % 1000 == 0:
        save_path = saver.save(sess, savePath + 'model', global_step=iteration)
        print("Model saved in file: %s" % save_path)

2017-11-05 17:39:03: Accuracy at step 0: 4.16288
Model saved in file: ./saves.tf.Mines3/(8, 8)/model-0
2017-11-05 17:39:05: Accuracy at step 10: 4.15359
2017-11-05 17:39:06: Accuracy at step 20: 4.14428
2017-11-05 17:39:08: Accuracy at step 30: 4.13688
2017-11-05 17:39:09: Accuracy at step 40: 4.13064
2017-11-05 17:39:10: Accuracy at step 50: 4.1244
2017-11-05 17:39:12: Accuracy at step 60: 4.12026
2017-11-05 17:39:13: Accuracy at step 70: 4.11397
2017-11-05 17:39:15: Accuracy at step 80: 4.10943
2017-11-05 17:39:16: Accuracy at step 90: 4.10581
2017-11-05 17:39:17: Accuracy at step 100: 4.10156
2017-11-05 17:39:19: Accuracy at step 110: 4.10029
2017-11-05 17:39:20: Accuracy at step 120: 4.09381
2017-11-05 17:39:21: Accuracy at step 130: 4.09285
2017-11-05 17:39:23: Accuracy at step 140: 4.08786
2017-11-05 17:39:24: Accuracy at step 150: 4.08756
2017-11-05 17:39:26: Accuracy at step 160: 4.08354
2017-11-05 17:39:27: Accuracy at step 170: 4.07956
2017-11-05 17:39:28: Accuracy at step 18

KeyboardInterrupt: 

In [17]:
# Test trained model
batch_xs, batch_ys = next_training_batch(1000)
print(sess.run(cross_entropy, feed_dict={mineCounts: batch_xs, mineFreeAverages: batch_ys}))

4.04015


In [19]:
# Run a test
batchSize = 100000
batch_xs, batch_ys = next_training_batch(batchSize)

predictions = sess.run(tf.nn.softmax(y), feed_dict={mineCounts: batch_xs, mineFreeAverages: batch_ys})
bestSquares = [pred.argmax() for pred in predictions]
board = (batch_ys == 0).astype(int)
frees = [board[i][bestSquares[i]] for i in range(batchSize)]
print("Number of errors for batch size of ", batchSize)
print(sum(frees))

Number of errors for batch size of  100000
0
